In [3]:
import pandas as pd 
import numpy as np 
import time 
import gc 
np.random.seed(42)

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

from keras.models import Model
from keras.layers import Input, Dropout, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, BatchNormalization, Conv1D, MaxPooling1D, Flatten
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import backend as K
from keras.models import Model


from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import backend as K
from keras.models import Model

from keras import optimizers

from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler

import warnings
warnings.filterwarnings('ignore')

import os
os.environ['OMP_NUM_THREADS'] = '4'

import threading
import multiprocessing
from multiprocessing import Pool, cpu_count
from contextlib import closing
cores = 4

from keras import backend as K
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

### rmse loss for keras
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_true - y_pred))) 

Using TensorFlow backend.


In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
gp = pd.read_csv('aggregated_features.csv') 
train = train.merge(gp, on='user_id', how='left')
test = test.merge(gp, on='user_id', how='left')

agg_cols = list(gp.columns)[1:]

del gp
gc.collect()

train.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,...,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability,avg_days_up_user,avg_times_up_user,n_user_items
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),...,400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789,8.000000,2.000000,2
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,...,3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000,NaN,NaN,1
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,...,4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177,4.428571,1.142857,9
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,...,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323,16.714286,2.642857,32
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",...,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797,NaN,NaN,1


In [5]:
#image_confidence: keras pretrained inception model

confidence = pd.read_csv('results/total_result.csv', usecols=["image", "image_confidence"]) 
confidence1 = pd.read_csv('results/total_result1.csv', usecols=["image", "image_confidence"]) 
train = train.merge(confidence, on='image', how='left')
test = test.merge(confidence1, on='image', how='left')
train.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,...,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability,avg_days_up_user,avg_times_up_user,n_user_items,image_confidence
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),...,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789,8.000000,2.000000,2,0.906962
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,...,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000,NaN,NaN,1,0.635181
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,...,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177,4.428571,1.142857,9,0.720701
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,...,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323,16.714286,2.642857,32,0.413587
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",...,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797,NaN,NaN,1,0.288239


In [7]:
test.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,...,price,item_seq_number,activation_date,user_type,image,image_top_1,avg_days_up_user,avg_times_up_user,n_user_items,image_confidence
0,6544e41a8817,dbe73ad6e4b5,Волгоградская область,Волгоград,Личные вещи,Детская одежда и обувь,Для мальчиков,Обувь,25,Отдам бесплатно,...,NaN,66,2017-04-18,Private,a8b57acb5ab304f9c331ac7a074219aed4d349d8aef386...,2020.0,7.500000,1.500000,6,0.956819
1,65b9484d670f,2e11806abe57,Свердловская область,Нижняя Тура,Хобби и отдых,Велосипеды,Дорожные,NaN,NaN,Продам велосипед,...,3000.0,4,2017-04-16,Private,NaN,NaN,NaN,NaN,1,NaN
2,8bab230b2ecd,0b850bbebb10,Новосибирская область,Бердск,Бытовая электроника,Аудио и видео,Телевизоры и проекторы,NaN,NaN,BBK,...,15000.0,15,2017-04-17,Private,8c361112cb049745ef2d1b0ae73594fc5c107286b0c942...,2960.0,6.000000,1.000000,4,0.931989
3,8e348601fefc,5f1d5c3ce0da,Саратовская область,Саратов,Для дома и дачи,Бытовая техника,Для кухни,Вытяжки,NaN,Вытяжка Jetair 60,...,4500.0,70,2017-04-17,Private,NaN,NaN,15.066667,2.666667,19,NaN
4,8bd2fe400b89,23e2d97bfc7f,Оренбургская область,Бузулук,Личные вещи,Товары для детей и игрушки,Детские коляски,NaN,NaN,Коляска зима-лето,...,4900.0,15,2017-04-15,Private,bc3cf6deef10840fc302e38eb48fa7748aa1e28d534f8f...,1002.0,6.500000,1.000000,5,0.699489


In [10]:
city = pd.read_csv('avito_region_city_features.csv') 
train = train.merge(city, on=['region', 'city'], how='left')
test = test.merge(city, on=['region', 'city'], how='left')


del city
gc.collect()

35

In [11]:
train.columns

Index(['item_id', 'user_id', 'region', 'city', 'parent_category_name',
       'category_name', 'param_1', 'param_2', 'param_3', 'title',
       'description', 'price', 'item_seq_number', 'activation_date',
       'user_type', 'image', 'image_top_1', 'deal_probability',
       'avg_days_up_user', 'avg_times_up_user', 'n_user_items',
       'image_confidence', 'city_region', 'latitude', 'longitude',
       'lat_lon_hdbscan_cluster_05_03', 'lat_lon_hdbscan_cluster_10_03',
       'lat_lon_hdbscan_cluster_20_03', 'region_id', 'city_region_id'],
      dtype='object')

In [12]:
#填補image_top_1缺失值
train_img = pd.read_csv('train_image_top_1_features.csv') 
test_img = pd.read_csv('test_image_top_1_features.csv')
train.drop(['image_top_1'], axis=1, inplace=True)
test.drop(['image_top_1'], axis=1, inplace=True)
train

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,...,n_user_items,image_confidence,city_region,latitude,longitude,lat_lon_hdbscan_cluster_05_03,lat_lon_hdbscan_cluster_10_03,lat_lon_hdbscan_cluster_20_03,region_id,city_region_id
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),...,2,0.906962,Екатеринбург Свердловская область,56.838926,60.605702,65,38,21,19,477
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,...,1,0.635181,Самара Самарская область,53.241504,50.221246,58,36,19,17,1370
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,...,9,0.720701,Ростов-на-Дону Ростовская область,47.235714,39.701505,46,40,9,16,1346
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,...,32,0.413587,Набережные Челны Татарстан,55.718505,52.372104,55,33,18,21,982
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",...,1,0.288239,Волгоград Волгоградская область,48.708048,44.513303,31,17,-1,4,327
5,51e0962387f7,bbfad0b1ad0a,Татарстан,Чистополь,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Авто люлька,...,2,0.495085,Чистополь Татарстан,55.363473,50.640464,-1,-1,-1,21,1743
6,c4f260a2b48a,08f469d2e6f7,Нижегородская область,Нижний Новгород,Для дома и дачи,Ремонт и строительство,Сантехника и сауна,NaN,NaN,Водонагреватель 100 литров нержавейка плоский,...,11,0.993330,Нижний Новгород Нижегородская область,56.296504,43.936059,3,2,1,11,1018
7,6b71309d6a8a,fef86baa002c,Пермский край,Пермь,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Джинсы,26,Бойфренды colins,...,23,0.999984,Пермь Пермский край,58.029681,56.266792,43,23,12,15,1213
8,c5b969cb63a2,055825270190,Оренбургская область,Оренбург,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,> 50 (XXL),Платье,...,38,0.485692,Оренбург Оренбургская область,51.766648,55.100454,49,27,15,14,1162
9,b1570962e68c,f9e8f831d94c,Нижегородская область,Нижний Новгород,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,25,Полу ботиночки замш натур.Бамбини,...,93,0.977974,Нижний Новгород Нижегородская область,56.296504,43.936059,3,2,1,11,1018


In [13]:
train = train.merge(train_img, on=['item_id'], how='left')
test = test.merge(test_img, on=['item_id'], how='left')

del train_img, test_img
gc.collect()

42

In [14]:
def preprocess(dataset):
    
    t1 = time.time()
    print("Filling Missing Values.....")
    
    dataset['image_confidence'] = dataset['image_confidence'].fillna(0).astype('float32')
    dataset['avg_days_up_user'] = dataset['avg_days_up_user'].fillna(0).astype('uint32')
    dataset['avg_times_up_user'] = dataset['avg_times_up_user'].fillna(0).astype('uint32')
    dataset['n_user_items'] = dataset['n_user_items'].fillna(0).astype('uint32')
    
    dataset['price'] = dataset['price'].fillna(0).astype('float32')
    dataset['param_1'].fillna(value='missing', inplace=True)
    dataset['param_2'].fillna(value='missing', inplace=True)
    dataset['param_3'].fillna(value='missing', inplace=True)
    
    #2018/11/10
    dataset['title'] = dataset['title'].astype(str)
    dataset['description'] = dataset['description'].astype(str)
    #2018/11/10
    dataset['param_1'] = dataset['param_1'].astype(str)
    dataset['param_2'] = dataset['param_2'].astype(str)
    dataset['param_3'] = dataset['param_3'].astype(str)
    
    print("Casting data types to type Category.......")
    dataset['category_name'] = dataset['category_name'].astype('category')
    dataset['parent_category_name'] = dataset['parent_category_name'].astype('category')
    dataset['region'] = dataset['region'].astype('category')
    dataset['city'] = dataset['city'].astype('category')
   
    dataset['param_1'] = dataset['param_1'].astype('category')
    dataset['param_2'] = dataset['param_2'].astype('category')
    dataset['param_3'] = dataset['param_3'].astype('category')    
    
    dataset['image_top_1'] = dataset['image_top_1'].fillna('missing')
    dataset['image_code'] = dataset['image_top_1'].astype('str')
    del dataset['image_top_1']
    gc.collect()

    #dataset['week'] = pd.to_datetime(dataset['activation_date']).dt.week.astype('uint8')
    #dataset['day'] = pd.to_datetime(dataset['activation_date']).dt.day.astype('uint8')
    #dataset['wday'] = pd.to_datetime(dataset['activation_date']).dt.dayofweek.astype('uint8')

    dataset['day'] = pd.to_datetime(dataset['activation_date']).dt.day.astype('uint8')    
    
    del dataset['activation_date']
    gc.collect()
    
    print("PreProcessing Function completed.")
    
    return dataset

In [15]:
train = preprocess(train)
test = preprocess(test)

Filling Missing Values.....
Casting data types to type Category.......
PreProcessing Function completed.
Filling Missing Values.....
Casting data types to type Category.......
PreProcessing Function completed.


In [23]:
submission1 = pd.DataFrame( columns = ['item_id', 'deal_probability'])
submission1['item_id'] = test['item_id']
df = pd.concat([train, test], axis = 0)
gc.collect()

In [28]:
all_text = np.hstack([df['title'].str.lower()])
all_text1 = np.hstack([df['description'].str.lower()])

df['title'] = df['title'].astype(str)
df['description'] = df['description'].astype(str)

In [31]:
tokenizer = text.Tokenizer(num_words = 50000)
tokenizer.fit_on_texts(all_text)

tokenizer1 = text.Tokenizer(num_words = 100000)
tokenizer1.fit_on_texts(all_text1)

sequence = tokenizer.texts_to_sequences(df.title.str.lower())
sequence1 = tokenizer1.texts_to_sequences(df.description.str.lower())

pad_sequence=pad_sequences(sequence, maxlen=15) 
pad_sequence1=pad_sequences(sequence1, maxlen=50) 

In [36]:
del sequence, sequence1, all_text, all_text1
gc.collect()

0

In [37]:
print("Start Label Encoding process....")
le_region = LabelEncoder()
df['region'] = le_region.fit_transform(df.region)
    
le_city = LabelEncoder()
df['city'] = le_city.fit_transform(df.city)
    
le_category_name = LabelEncoder()
df['category_name'] = le_category_name.fit_transform(df.category_name)
    
le_parent_category_name = LabelEncoder()
df['parent_category_name'] = le_parent_category_name.fit_transform(df.parent_category_name)
    
le_image_code = LabelEncoder()
df['image_code'] = le_image_code.fit_transform(df.image_code)

le_user_type = LabelEncoder()
df['user_type'] = le_user_type.fit_transform(df.user_type)

le_param1 = LabelEncoder()
df['param_1'] = le_param1.fit_transform(df.param_1)

le_param2 = LabelEncoder()
df['param_2'] = le_param2.fit_transform(df.param_2)

le_param3 = LabelEncoder()
df['param_3'] = le_param3.fit_transform(df.param_3)

Start Label Encoding process....


In [38]:
del df['image'], df['user_id'], df['item_id'], df['city_region'], df['description'], df['title'], df['region_id']
gc.collect()

72

In [41]:
df['price'] = np.log1p(df['price'])
df['avg_days_up_user'] = np.log1p(df['avg_days_up_user'])
df['avg_times_up_user'] = np.log1p(df['avg_times_up_user'])
df['n_user_items'] = np.log1p(df['n_user_items'])
df['item_seq_number'] = np.log(df['item_seq_number'])
print("Fit on Train Function completed.")

Fit on Train Function completed.


In [47]:
max_features = 50000
embed_size = 300

def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open('cc.ru.300.vec/cc.ru.300.vec', encoding = 'utf8'))

word_index = tokenizer.word_index
nb_words = min(50000, len(word_index))
embedding_matrix = np.zeros((nb_words, 300))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [48]:
max_features = 100000
embed_size = 300

def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')

word_index1 = tokenizer1.word_index
nb_words = min(100000, len(word_index1))
embedding_matrix1 = np.zeros((nb_words, 300))
for word, i in word_index1.items():
    if i >= max_features: continue
    embedding_vector1 = embeddings_index.get(word)
    if embedding_vector1 is not None: embedding_matrix1[i] = embedding_vector1

In [52]:
df.columns

Index(['avg_days_up_user', 'avg_times_up_user', 'category_name', 'city',
       'city_region_id', 'day', 'deal_probability', 'image_code',
       'image_confidence', 'item_seq_number', 'lat_lon_hdbscan_cluster_05_03',
       'lat_lon_hdbscan_cluster_10_03', 'lat_lon_hdbscan_cluster_20_03',
       'latitude', 'longitude', 'n_user_items', 'param_1', 'param_2',
       'param_3', 'parent_category_name', 'price', 'region', 'user_type'],
      dtype='object')

In [54]:
train = df[:1503424]
test = df[1503424:]

In [55]:
y_train = np.array(train['deal_probability'])
del train['deal_probability'], test['deal_probability']
gc.collect()

In [57]:
pad_sequence_train = pad_sequence[:1503424]
pad_sequence_test = pad_sequence[1503424:]

pad_sequence_train1 = pad_sequence1[:1503424]
pad_sequence_test1 = pad_sequence1[1503424:]

In [59]:
pad_sequence_train.shape

(1503424, 15)

In [62]:
    def data_transform(train, pad_sequence, pad_sequence1):
        data1 = {
        'seq_title': pad_sequence
        ,'seq_description': pad_sequence1
                    ,'param_1': np.array(train.param_1)
                    ,'param_2': np.array(train.param_2)
                    ,'param_3': np.array(train.param_3)
        ,'region': np.array(train.region)
        ,'city': np.array(train.city)
        ,'city_region_id': np.array(train.city_region_id)
        ,'day': np.array(train.day)
        ,'category_name': np.array(train.category_name)
        ,'parent_category_name': np.array(train.parent_category_name)
        ,'image_code':np.array(train.image_code)
            ,'lat_lon_hdbscan_cluster_05_03':np.array(train.lat_lon_hdbscan_cluster_05_03)
            ,'lat_lon_hdbscan_cluster_10_03':np.array(train.lat_lon_hdbscan_cluster_10_03)
            ,'lat_lon_hdbscan_cluster_20_03':np.array(train.lat_lon_hdbscan_cluster_20_03)
        ,'price': np.array(train[["price"]])
            ,'latitude': np.array(train[["latitude"]])
            ,'longitude': np.array(train[["longitude"]])
        ,'item_seq_number': np.array(train[["item_seq_number"]])
        ,'avg_days_up_user': np.array(train[["avg_days_up_user"]])
        ,'avg_times_up_user': np.array(train[["avg_times_up_user"]])
        ,'n_user_items': np.array(train[["n_user_items"]])
        ,'user_type': np.array(train[["user_type"]])
        ,'image_confidence': np.array(train[["image_confidence"]])
    }
        print("Data ready for Vectorization")
    
        return data1

In [63]:
final_train = data_transform(train, pad_sequence_train, pad_sequence_train1)
final_test = data_transform(test, pad_sequence_test, pad_sequence_test1)

Data ready for Vectorization
Data ready for Vectorization


In [65]:
final_train.keys()

dict_keys(['seq_title', 'seq_description', 'param_1', 'param_2', 'param_3', 'region', 'city', 'city_region_id', 'day', 'category_name', 'parent_category_name', 'image_code', 'lat_lon_hdbscan_cluster_05_03', 'lat_lon_hdbscan_cluster_10_03', 'lat_lon_hdbscan_cluster_20_03', 'price', 'latitude', 'longitude', 'item_seq_number', 'avg_days_up_user', 'avg_times_up_user', 'n_user_items', 'user_type', 'image_confidence'])

In [66]:
from keras.layers.advanced_activations import PReLU

#feature input
seq_title = Input(shape=[15], name="seq_title")
seq_description = Input(shape=[50], name="seq_description")
param_1 = Input(shape=[1], name="param_1")
param_2 = Input(shape=[1], name="param_2")
param_3 = Input(shape=[1], name="param_3")
region = Input(shape=[1], name="region")
city = Input(shape=[1], name="city")
city_region_id = Input(shape=[1], name="city_region_id")
day = Input(shape=[1], name="day")
category_name = Input(shape=[1], name="category_name")
parent_category_name = Input(shape=[1], name="parent_category_name")
image_code = Input(shape=[1], name="image_code")
lat_lon_hdbscan_cluster_05_03 = Input(shape=[1], name="lat_lon_hdbscan_cluster_05_03")
lat_lon_hdbscan_cluster_10_03 = Input(shape=[1], name="lat_lon_hdbscan_cluster_10_03")
lat_lon_hdbscan_cluster_20_03 = Input(shape=[1], name="lat_lon_hdbscan_cluster_20_03")
price = Input(shape=[1], name="price")
latitude = Input(shape=[1], name="latitude")
longitude = Input(shape=[1], name="longitude")
item_seq_number = Input(shape = [1], name = 'item_seq_number')
avg_days_up_user = Input(shape=[1], name="avg_days_up_user")
avg_times_up_user = Input(shape=[1], name="avg_times_up_user")
n_user_items = Input(shape=[1], name="n_user_items")
user_type = Input(shape=[1], name="user_type")
image_confidence = Input(shape=[1], name="image_confidence")

#embedding layer:
emb_seq_title = Embedding(50000, 300, weights = [embedding_matrix], trainable = False)(seq_title)
emb_seq_description = Embedding(100000, 300, weights = [embedding_matrix1], trainable = False)(seq_description)

emb_param_1 = Embedding(50000, 300)(param_1)
emb_param_2 = Embedding(50000, 300)(param_2)
emb_param_3 = Embedding(50000, 300)(param_3)

emb_region = Embedding(50000, 10)(region)
emb_city = Embedding(50000, 10)(city)
emb_city_region_id = Embedding(50000, 10)(city_region_id)
emb_day = Embedding(50000, 10)(day)
emb_category_name = Embedding(50000, 10)(category_name)
emb_parent_category_name = Embedding(50000, 10)(parent_category_name)
emb_image_code = Embedding(50000, 10)(image_code)
emb_lat_lon_hdbscan_cluster_05_03 = Embedding(50000, 10)(lat_lon_hdbscan_cluster_05_03)
emb_lat_lon_hdbscan_cluster_10_03 = Embedding(50000, 10)(lat_lon_hdbscan_cluster_10_03)
emb_lat_lon_hdbscan_cluster_20_03 = Embedding(50000, 10)(lat_lon_hdbscan_cluster_20_03)

# GRU layer>title/description
rnn_layer1 = GRU(128, dropout=0.1) (emb_seq_title)
rnn_layer1 = Dense(256) (rnn_layer1)
rnn_layer1 = PReLU()(rnn_layer1)
rnn_layer1 = Dropout(0.1)(rnn_layer1)

rnn_layer2 = GRU(128, dropout=0.1) (emb_seq_description)
rnn_layer2 = Dense(256) (rnn_layer2)
rnn_layer2 = PReLU()(rnn_layer2)
rnn_layer2 = Dropout(0.1)(rnn_layer2)

#concatenate text input (title/description)
main_a = concatenate([rnn_layer1, rnn_layer2])
main_a = Dense(256) (main_a)
main_a = PReLU()(main_a)
main_a = Dropout(0.1)(main_a)

#concatenate categorical input
main_b = concatenate([
          Flatten() (emb_param_1)
        , Flatten() (emb_param_2)
        , Flatten() (emb_param_3)
        , Flatten() (emb_region)
        , Flatten() (emb_city)
        , Flatten() (emb_city_region_id)
        , Flatten() (emb_day)
        , Flatten() (emb_category_name)
        , Flatten() (emb_parent_category_name)
        , Flatten() (emb_image_code)
#        , Flatten() (emb_lat_lon_hdbscan_cluster_05_03)
#        , Flatten() (emb_lat_lon_hdbscan_cluster_10_03)
#        , Flatten() (emb_lat_lon_hdbscan_cluster_20_03)
    ])
main_b = Dense(256) (main_b)
main_b = PReLU()(main_b)
main_b = Dropout(0.1)(main_b)

#concatenate numerical input
main_c = concatenate([
         avg_days_up_user
        , avg_times_up_user
        , n_user_items
        , price
        , item_seq_number
            , image_confidence
    ])
main_c = Dense(256) (main_c)
main_c = PReLU()(main_c)
main_c = Dropout(0.1)(main_c)

 
#main layer 1 
main_l = concatenate([
            main_a, main_b, main_c])

main_l = Dense(512) (main_l)
main_l = PReLU()(main_l)
main_l = Dropout(0.1)(main_l)

#main layer 2
main_l = Dense(64) (main_l)
main_l = PReLU()(main_l)
main_l = Dropout(0.1)(main_l)

#output
output = Dense(1,activation="sigmoid") (main_l)
    
#model
model = Model([seq_title, seq_description, param_1, param_2, param_3, region, city, city_region_id, day, category_name, parent_category_name, image_code, lat_lon_hdbscan_cluster_05_03, lat_lon_hdbscan_cluster_10_03, lat_lon_hdbscan_cluster_20_03, price, latitude, longitude, item_seq_number, avg_days_up_user, avg_times_up_user, n_user_items, user_type, image_confidence], output)
model.compile(optimizer = 'adam',
                  loss= root_mean_squared_error,
                  metrics = [root_mean_squared_error])

In [68]:
gc.collect()

174

In [73]:
final_train.keys()

dict_keys(['seq_title', 'seq_description', 'param_1', 'param_2', 'param_3', 'region', 'city', 'city_region_id', 'day', 'category_name', 'parent_category_name', 'image_code', 'lat_lon_hdbscan_cluster_05_03', 'lat_lon_hdbscan_cluster_10_03', 'lat_lon_hdbscan_cluster_20_03', 'price', 'latitude', 'longitude', 'item_seq_number', 'avg_days_up_user', 'avg_times_up_user', 'n_user_items', 'user_type', 'image_confidence'])

In [ ]:
model.fit(final_train, y_train, batch_size=64, epochs=10, validation_split=0.2, verbose=1)


In [ ]:
y = model.predict(final_test, batch_size = 512, verbose = 1)
submission1['deal_probability'] = y
submission1.to_csv("Avito-Demand-Prediction-Challenge_2018_11_12_28th_edition.csv", index=False)